In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import plotly.express as px

%matplotlib inline

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
county = pd.read_csv('../Data/master_county.csv')

In [ ]:
county

In [ ]:
for index, row in county.iterrows():
    if row.Rep > row.Dem:
        county.loc[index, 'Rep_win'] = 1
    elif row.Rep < row.Dem:
        county.loc[index, 'Rep_win'] = 0

In [ ]:
for index, row in county.iterrows():
    if row.Dem > row.Rep:
        county.loc[index, 'Dem_win'] = 1
    elif row.Dem < row.Rep:
        county.loc[index, 'Dem_win'] = 0

In [ ]:
county_split = county.groupby(['year', 'county']).agg({'Rep_win': 'sum', 'Dem_win': 'sum'}).reset_index()

In [ ]:
county_split.head(3)

In [ ]:
county_split = county_split.astype({'Rep_win': 'int', 'Dem_win': 'int'})

In [ ]:
for index, row in county_split.iterrows():
    if row.Rep_win == 0:
        county_split.loc[index, 'all_dem'] = 1
    elif row.Dem_win == 0:
        county_split.loc[index, 'all_rep'] = 1
    else:
        county_split.loc[index, 'split'] = 1

In [ ]:
for index, row in county_split.iterrows():
    if row.split == 1:
        county_split.loc[index, 'one_party'] = 0
    else:
        county_split.loc[index, 'one_party'] = 1

In [ ]:
county_split = county_split.fillna(0)

In [ ]:
county_split = county_split.astype({'split': 'int', 'all_dem': 'int', 'all_rep': 'int', 'one_party': 'int'})

In [ ]:
county_split.one_party.value_counts()

In [ ]:
county_split.split.value_counts()

In [ ]:
county_split.all_dem.value_counts()

In [ ]:
county_split.all_rep.value_counts()

In [ ]:
county_split

In [ ]:
county_tot = county_split.groupby(['year']).agg({'split': 'sum', 'all_dem': 'sum', 'all_rep': 'sum', 'one_party': 'sum'}).reset_index()

In [ ]:
county_tot.set_index('year', inplace=True)

In [ ]:
county_tot

In [ ]:
# county_tot.to_csv("split_county_by_year.csv", index=False)

In [ ]:
spl = county_tot['split']

ax = county_tot.sort_index(ascending=False)[['split', 'one_party']].plot(kind = 'bar',
                                                                    stacked = True, 
                                                                    align='center',
                                                                    width=0.75,
                                                                    colormap='tab20',
                                                                    edgecolor='black',
                                                                    linewidth=2,
                                                                    figsize=(8,4))



ax=plt.gca()
ax.invert_xaxis() 

for index, value in enumerate(spl):
  ax.text(index, value, str(value), fontsize=16, ha='center')

plt.legend(bbox_to_anchor = (1, 0.7), loc = 'upper left')
plt.xlabel('Year', fontsize=16, fontweight='bold')
plt.ylabel('Number of Counties', fontsize=16, fontweight='bold')
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14)
plt.title('Split Ticket Counties by Year', fontsize=18, fontweight='bold', pad=20)

# plt.savefig('split_counties.png', bbox_inches='tight');
plt.show();

In [ ]:
plt.figure(figsize = (10,6))
sns.set_theme(style="darkgrid")

graph = sns.lineplot(data = county_tot,
               x = 'year',
               y = 'split',
               palette = 'tab10',
               linewidth = 3,
               marker = 'o', markersize=9)
# graph.axhline(50, color='black', linestyle = '--', linewidth = 2)
# plt.legend(bbox_to_anchor = (1, 1), loc = 'upper left')
plt.title('Split Ticket Counties by Year', fontsize=16, fontweight='bold');

In [ ]:
county_10 = county[county['year'] < 2010]
county_20 = county[county['year'] >= 2010]

In [ ]:
county_10

In [ ]:
county_20

In [ ]:
county_20['office'] = county_20['office'].replace({'State House' : 'State_House'}, regex=True)

In [ ]:
ps20 = county_20[['year', 'county', 'office', 'Rep%', 'Dem%']]

In [ ]:
mask = ps20['office'].isin(['Pres', 'US_Senate'])

In [ ]:
ps20 = ps20[mask]

In [ ]:
ps20

In [ ]:
ps20 = ps20.pivot_table(index=['year', 'county'],
                 columns = 'office',
                 values = ['Rep%', 'Dem%']).reset_index()

In [ ]:
ps20

In [ ]:
plt.figure(figsize=(8,8), dpi=80)
plt.scatter(ps20['Rep%'].Pres, ps20['Rep%'].US_Senate, s=50, c='red', edgecolor='black', linewidth=0.7)

plt.title('Republican Vote Share by County \n Presidential vs. US Senate, 2010-2020', fontweight = 'bold', fontsize = 16, color = 'black')
plt.xlabel('Republican % by County, Presidential', fontweight = 'bold', fontsize = 14, color = 'black')
plt.ylabel('Republican % by County, US Senate', fontweight = 'bold', fontsize = 14, color = 'black')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.xlim(10, 90)
plt.ylim(10, 90)
plt.axhline(y=50, color='black', linestyle = '--', linewidth=2)
plt.axvline(x=50, color='black', linestyle = '--', linewidth=2)
ax = plt.gca()
ax.set_facecolor('lightgrey')

plt.show()

In [ ]:
plt.figure(figsize=(8,8), dpi=80)
plt.scatter(ps20['Dem%'].Pres, ps20['Dem%'].US_Senate, s=50, c='royalblue', edgecolor='black', linewidth=0.7)

plt.title('Democratic Vote Share by County \n Presidential vs. US Senate, 2010-2020', fontweight = 'bold', fontsize = 16, color = 'black')
plt.xlabel('Democratic % by County, Presidential', fontweight = 'bold', fontsize = 14, color = 'black')
plt.ylabel('Democratic % by County, US Senate', fontweight = 'bold', fontsize = 14, color = 'black')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.xlim(8, 92)
plt.ylim(8, 92)
plt.axhline(y=50, color='black', linestyle = '--', linewidth=2)
plt.axvline(x=50, color='black', linestyle = '--', linewidth=2)
ax = plt.gca()
ax.set_facecolor('lightgrey')

plt.show()

In [ ]:
sg20 = county_20[['year', 'county', 'office', 'Rep%', 'Dem%']]

In [ ]:
sgmask = sg20['office'].isin(['US_Senate', 'Governor'])

In [ ]:
sg20 = sg20[sgmask]

In [ ]:
sg20

In [ ]:
sg20 = sg20.pivot_table(index=['year', 'county'],
                 columns = 'office',
                 values = ['Rep%', 'Dem%']).reset_index()

In [ ]:
sg20

In [ ]:
plt.figure(figsize=(8,8), dpi=80)
plt.scatter(sg20['Rep%'].Governor, sg20['Rep%'].US_Senate, s=50, c='red', edgecolor='black', linewidth=0.7)

plt.title('Republican Vote Share by County \n Gubernatorial vs. US Senate, 2010-2020', fontweight = 'bold', fontsize = 16, color = 'black')
plt.xlabel('Republican % by County, Gubernatorial', fontweight = 'bold', fontsize = 14, color = 'black')
plt.ylabel('Republican % by County, US Senate', fontweight = 'bold', fontsize = 14, color = 'black')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.xlim(10, 90)
plt.ylim(10, 90)
plt.axhline(y=50, color='black', linestyle = '--', linewidth=2)
plt.axvline(x=50, color='black', linestyle = '--', linewidth=2)
ax = plt.gca()
ax.set_facecolor('lightgrey')

plt.show()

In [ ]:
plt.figure(figsize=(8,8), dpi=80)
plt.scatter(sg20['Dem%'].Governor, sg20['Dem%'].US_Senate, s=50, c='royalblue', edgecolor='black', linewidth=0.7)

plt.title('Democratic Vote Share by County \n Gubernatorial vs. US Senate, 2010-2020', fontweight = 'bold', fontsize = 16, color = 'black')
plt.xlabel('Democratic % by County, Gubernatorial', fontweight = 'bold', fontsize = 14, color = 'black')
plt.ylabel('Democratic % by County, US Senate', fontweight = 'bold', fontsize = 14, color = 'black')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.xlim(10, 90)
plt.ylim(10, 90)
plt.axhline(y=50, color='black', linestyle = '--', linewidth=2)
plt.axvline(x=50, color='black', linestyle = '--', linewidth=2)
ax = plt.gca()
ax.set_facecolor('lightgrey')

plt.show()

In [ ]:
ph20 = county_20[['year', 'county', 'office', 'Rep%', 'Dem%']]

In [ ]:
phmask = ph20['office'].isin(['Pres', 'US_House'])

In [ ]:
ph20 = ph20[phmask]

In [ ]:
ph20

In [ ]:
ph20 = ph20.pivot_table(index=['year', 'county'],
                 columns = 'office',
                 values = ['Rep%', 'Dem%']).reset_index()

In [ ]:
ph20

In [ ]:
plt.figure(figsize=(8,8), dpi=80)
plt.scatter(ph20['Rep%'].Pres, ph20['Rep%'].US_House, s=50, c='red', edgecolor='black', linewidth=0.7)

plt.title('Republican Vote Share by County \n Presidential vs. US House, 2010-2020', fontweight = 'bold', fontsize = 16, color = 'black')
plt.xlabel('Republican % by County, Presidential', fontweight = 'bold', fontsize = 14, color = 'black')
plt.ylabel('Republican % by County, US House', fontweight = 'bold', fontsize = 14, color = 'black')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.xlim(10, 90)
plt.ylim(10, 90)
plt.axhline(y=50, color='black', linestyle = '--', linewidth=2)
plt.axvline(x=50, color='black', linestyle = '--', linewidth=2)
ax = plt.gca()
ax.set_facecolor('lightgrey')

plt.show()

In [ ]:
plt.figure(figsize=(8,8), dpi=80)
plt.scatter(ph20['Dem%'].Pres, ph20['Dem%'].US_House, s=50, c='royalblue', edgecolor='black', linewidth=0.7)

plt.title('Democratic Vote Share by County \n Presidential vs. US House, 2010-2020', fontweight = 'bold', fontsize = 16, color = 'black')
plt.xlabel('Democratic % by County, Presidential', fontweight = 'bold', fontsize = 14, color = 'black')
plt.ylabel('Democratic % by County, US House', fontweight = 'bold', fontsize = 14, color = 'black')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.xlim(10, 90)
plt.ylim(10, 90)
plt.axhline(y=50, color='black', linestyle = '--', linewidth=2)
plt.axvline(x=50, color='black', linestyle = '--', linewidth=2)
ax = plt.gca()
ax.set_facecolor('lightgrey')

plt.show()

In [ ]:
gh20 = county_20[['year', 'county', 'office', 'Rep%', 'Dem%']]

In [ ]:
ghmask = gh20['office'].isin(['Governor', 'US_House'])

In [ ]:
gh20 = gh20[ghmask]

In [ ]:
gh20

In [ ]:
gh20 = gh20.pivot_table(index=['year', 'county'],
                 columns = 'office',
                 values = ['Rep%', 'Dem%']).reset_index()

In [ ]:
gh20

In [ ]:
plt.figure(figsize=(8,8), dpi=80)
plt.scatter(gh20['Rep%'].Governor, gh20['Rep%'].US_House, s=50, c='red', edgecolor='black', linewidth=0.7)

plt.title('Republican Vote Share by County \n Gubernatorial vs. US House, 2010-2020', fontweight = 'bold', fontsize = 16, color = 'black')
plt.xlabel('Republican % by County, Gubernatorial', fontweight = 'bold', fontsize = 14, color = 'black')
plt.ylabel('Republican % by County, US House', fontweight = 'bold', fontsize = 14, color = 'black')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.xlim(5, 95)
plt.ylim(5, 95)
plt.axhline(y=50, color='black', linestyle = '--', linewidth=2)
plt.axvline(x=50, color='black', linestyle = '--', linewidth=2)
ax = plt.gca()
ax.set_facecolor('lightgrey')

plt.show()

In [ ]:
plt.figure(figsize=(8,8), dpi=80)
plt.scatter(gh20['Dem%'].Governor, gh20['Dem%'].US_House, s=50, c='royalblue', edgecolor='black', linewidth=0.7)

plt.title('Democratic Vote Share by County \n Gubernatorial vs. US House, 2010-2020', fontweight = 'bold', fontsize = 16, color = 'black')
plt.xlabel('Democratic % by County, Gubernatorial', fontweight = 'bold', fontsize = 14, color = 'black')
plt.ylabel('Democratic % by County, US House', fontweight = 'bold', fontsize = 14, color = 'black')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.xlim(5, 95)
plt.ylim(5, 95)
plt.axhline(y=50, color='red', linestyle = '--', linewidth=2)
plt.axvline(x=50, color='red', linestyle = '--', linewidth=2)
ax = plt.gca()
ax.set_facecolor('lightgrey')

plt.show()

In [ ]:
plt.figure(figsize = (8,8))
sns.set_theme(style="darkgrid")

x = gh20['Rep%'].Governor
y = gh20['Rep%'].US_House

graph = sns.regplot(data = gh20, x=x, y=y, color='red', scatter_kws={'s':40, 'edgecolor':'black'}) 
graph.axhline(50, color='black', linestyle = '--', linewidth = 2)
graph.axvline(50, color='black', linestyle = '--', linewidth = 2)

plt.title('Republican Vote Share by County \n Gubernatorial vs. US House, 2010-2020', fontweight = 'bold', fontsize = 16, color = 'black')
plt.xlabel('Gubernatorial, Republican % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.ylabel('US House, Republican % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.xlim(10, 90)
plt.ylim(10, 90)
# plt.savefig('rep_gov_ush_20.png', bbox_inches='tight')
plt.plot();

In [ ]:
plt.figure(figsize = (8,8))
sns.set_theme(style="darkgrid")

x = gh20['Dem%'].Governor
y = gh20['Dem%'].US_House

graph = sns.regplot(data = gh20, x=x, y=y, color='royalblue', scatter_kws={'s':40, 'edgecolor':'black'}) 
graph.axhline(50, color='red', linestyle = '--', linewidth = 2)
graph.axvline(50, color='red', linestyle = '--', linewidth = 2)

plt.title('Democratic Vote Share by County \n Gubernatorial vs. US House, 2010-2020', fontweight = 'bold', fontsize = 16, color = 'black')
plt.xlabel('Gubernatorial, Democratic % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.ylabel('US House, Democratic % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.xlim(10, 90)
plt.ylim(10, 90)
# plt.savefig('dem_gov_ush_20.png', bbox_inches='tight')
plt.plot();

In [ ]:
plt.figure(figsize = (8,8))
sns.set_theme(style="darkgrid")

x = ph20['Rep%'].Pres
y = ph20['Rep%'].US_House

graph = sns.regplot(data = ph20, x=x, y=y, color='red', scatter_kws={'s':40, 'edgecolor':'black'}) 
graph.axhline(50, color='black', linestyle = '--', linewidth = 2)
graph.axvline(50, color='black', linestyle = '--', linewidth = 2)

plt.title('Republican Vote Share by County \n Presidential vs. US House, 2010-2020', fontweight = 'bold', fontsize = 16, color = 'black')
plt.xlabel('Presidential, Republican % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.ylabel('US House, Republican % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.xlim(10, 90)
plt.ylim(10, 90)
# plt.savefig('rep_pres_ush_20.png', bbox_inches='tight')
plt.plot();

In [ ]:
plt.figure(figsize = (8,8))
sns.set_theme(style="darkgrid")

x = ph20['Dem%'].Pres
y = ph20['Dem%'].US_House

graph = sns.regplot(data = ph20, x=x, y=y, color='royalblue', scatter_kws={'s':40, 'edgecolor':'black'}) 
graph.axhline(50, color='red', linestyle = '--', linewidth = 2)
graph.axvline(50, color='red', linestyle = '--', linewidth = 2)

plt.title('Democratic Vote Share by County \n Presidential vs. US House, 2010-2020', fontweight = 'bold', fontsize = 16, color = 'black')
plt.xlabel('Presidential, Democratic % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.ylabel('US House, Democratic % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.xlim(8, 92)
plt.ylim(8, 92)
# plt.savefig('dem_pres_ush_20.png', bbox_inches='tight')
plt.plot();

In [ ]:
plt.figure(figsize = (8,8))
sns.set_theme(style="darkgrid")

x = ps20['Rep%'].Pres
y = ps20['Rep%'].US_Senate

graph = sns.regplot(data = ps20, x=x, y=y, color='red', scatter_kws={'s':40, 'edgecolor':'black'}) 
graph.axhline(50, color='black', linestyle = '--', linewidth = 2)
graph.axvline(50, color='black', linestyle = '--', linewidth = 2)

plt.title('Republican Vote Share by County \n Presidential vs. US Senate, 2010-2020', fontweight = 'bold', fontsize = 16, color = 'black')
plt.xlabel('Presidential, Republican % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.ylabel('US Senate, Republican % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.xlim(10, 90)
plt.ylim(10, 90)
# plt.savefig('rep_pres_sen_20.png', bbox_inches='tight')
plt.plot();

In [ ]:
plt.figure(figsize = (8,8))
sns.set_theme(style="darkgrid")

x = ps20['Dem%'].Pres
y = ps20['Dem%'].US_Senate

graph = sns.regplot(data = ps20, x=x, y=y, color='royalblue', scatter_kws={'s':40, 'edgecolor':'black'}) 
graph.axhline(50, color='red', linestyle = '--', linewidth = 2)
graph.axvline(50, color='red', linestyle = '--', linewidth = 2)

plt.title('Democratic Vote Share by County \n Presidential vs. US Senate, 2010-2020', fontweight = 'bold', fontsize = 16, color = 'black')
plt.xlabel('Presidential, Democratic % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.ylabel('US Senate, Democratic % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.xlim(8, 92)
plt.ylim(8, 92)
# plt.savefig('dem_pres_sen_20.png', bbox_inches='tight')
plt.plot();

In [ ]:
plt.figure(figsize = (8,8))
sns.set_theme(style="darkgrid")

x = sg20['Rep%'].Governor
y = sg20['Rep%'].US_Senate

graph = sns.regplot(data = sg20, x=x, y=y, color='red', scatter_kws={'s':40, 'edgecolor':'black'}) 
graph.axhline(50, color='black', linestyle = '--', linewidth = 2)
graph.axvline(50, color='black', linestyle = '--', linewidth = 2)

plt.title('Republican Vote Share by County \n Gubernatorial vs. US Senate, 2010-2020', fontweight = 'bold', fontsize = 16, color = 'black')
plt.xlabel('Gubernatorial, Republican % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.ylabel('US Senate, Republican % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.xlim(10, 90)
plt.ylim(10, 90)
# plt.savefig('rep_gov_sen_20.png', bbox_inches='tight')
plt.plot();

In [ ]:
plt.figure(figsize = (8,8))
sns.set_theme(style="darkgrid")

x = sg20['Dem%'].Governor
y = sg20['Dem%'].US_Senate

graph = sns.regplot(data = sg20, x=x, y=y, color='royalblue', scatter_kws={'s':40, 'edgecolor':'black'}) 
graph.axhline(50, color='red', linestyle = '--', linewidth = 2)
graph.axvline(50, color='red', linestyle = '--', linewidth = 2)

plt.title('Democratic Vote Share by County \n Gubernatorial vs. US Senate, 2010-2020', fontweight = 'bold', fontsize = 16, color = 'black')
plt.xlabel('Gubernatorial, Democratic % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.ylabel('US Senate, Democratic % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.xlim(8, 92)
plt.ylim(8, 92)
# plt.savefig('dem_gov_sen_20.png', bbox_inches='tight')
plt.plot();

In [ ]:
pgh20 = county_20[['year', 'county', 'office', 'Rep%', 'Dem%']]

In [ ]:
pghmask = pgh20['office'].isin(['Pres', 'Governor', 'State_House'])

In [ ]:
pgh20 = pgh20[pghmask]

In [ ]:
pgh20

In [ ]:
pgh20 = pgh20.pivot_table(index=['year', 'county'],
                 columns = 'office',
                 values = ['Rep%', 'Dem%']).reset_index()

In [ ]:
pgh20.head()

In [ ]:
plt.figure(figsize = (8,8))
sns.set_theme(style="darkgrid")

x = pgh20['Rep%'].Pres
y = pgh20['Rep%'].State_House

graph = sns.regplot(data = pgh20, x=x, y=y, color='red', scatter_kws={'s':40, 'edgecolor':'black'}) 
graph.axhline(50, color='black', linestyle = '--', linewidth = 2)
graph.axvline(50, color='black', linestyle = '--', linewidth = 2)

plt.title('Republican Vote Share by County \n Presidential vs. TN House, 2010-2020', fontweight = 'bold', fontsize = 16, color = 'black')
plt.xlabel('Presidential, Republican % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.ylabel('TN House, Republican % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.xlim(8, 92)
plt.ylim(8, 92)
# plt.savefig('rep_pres_tnh_20.png', bbox_inches='tight')
plt.plot();

In [ ]:
plt.figure(figsize = (8,8))
sns.set_theme(style="darkgrid")

x = pgh20['Dem%'].Pres
y = pgh20['Dem%'].State_House

graph = sns.regplot(data = pgh20, x=x, y=y, color='royalblue', scatter_kws={'s':40, 'edgecolor':'black'}) 
graph.axhline(50, color='red', linestyle = '--', linewidth = 2)
graph.axvline(50, color='red', linestyle = '--', linewidth = 2)

plt.title('Democratic Vote Share by County \n Presidential vs. TN House, 2010-2020', fontweight = 'bold', fontsize = 16, color = 'black')
plt.xlabel('Presidential, Democratic % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.ylabel('TN House, Democratic % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.xlim(8, 92)
plt.ylim(8, 92)
# plt.savefig('dem_pres_tnh_20.png', bbox_inches='tight')
plt.plot();

In [ ]:
plt.figure(figsize = (8,8))
sns.set_theme(style="darkgrid")

x = pgh20['Rep%'].Governor
y = pgh20['Rep%'].State_House

graph = sns.regplot(data = pgh20, x=x, y=y, color='red', scatter_kws={'s':40, 'edgecolor':'black'}) 
graph.axhline(50, color='black', linestyle = '--', linewidth = 2)
graph.axvline(50, color='black', linestyle = '--', linewidth = 2)

plt.title('Republican Vote Share by County \n Gubernatorial vs. TN House, 2010-2020', fontweight = 'bold', fontsize = 16, color = 'black')
plt.xlabel('Gubernatorial, Republican % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.ylabel('TN House, Republican % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.xlim(8, 92)
plt.ylim(8, 92)
# plt.savefig('rep_gov_tnh_20.png', bbox_inches='tight')
plt.plot();

In [ ]:
plt.figure(figsize = (8,8))
sns.set_theme(style="darkgrid")

x = pgh20['Dem%'].Governor
y = pgh20['Dem%'].State_House

graph = sns.regplot(data = pgh20, x=x, y=y, color='royalblue', scatter_kws={'s':40, 'edgecolor':'black'}) 
graph.axhline(50, color='red', linestyle = '--', linewidth = 2)
graph.axvline(50, color='red', linestyle = '--', linewidth = 2)

plt.title('Democratic Vote Share by County \n Gubernatorial vs. TN House, 2010-2020', fontweight = 'bold', fontsize = 16, color = 'black')
plt.xlabel('Gubernatorial, Democratic % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.ylabel('TN House, Democratic % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.xlim(8, 92)
plt.ylim(8, 92)
# plt.savefig('dem_gov_tnh_20.png', bbox_inches='tight')
plt.plot();

In [ ]:
county_10['office'] = county_10['office'].replace({'State House' : 'State_House'}, regex=True)

In [ ]:
all10 = county_10[['year', 'county', 'office', 'Rep%', 'Dem%']]

In [ ]:
all10 = all10.pivot_table(index=['year', 'county'],
                 columns = 'office',
                 values = ['Rep%', 'Dem%']).reset_index()

In [ ]:
all10

In [ ]:
plt.figure(figsize = (8,8))
sns.set_theme(style="darkgrid")

x = all10['Rep%'].Pres
y = all10['Rep%'].US_Senate

graph = sns.regplot(data = all10, x=x, y=y, color='red', scatter_kws={'s':40, 'edgecolor':'black'}) 
graph.axhline(50, color='black', linestyle = '--', linewidth = 2)
graph.axvline(50, color='black', linestyle = '--', linewidth = 2)

plt.title('Republican Vote Share by County \n Presidential vs. US Senate, 2002-2008', fontweight = 'bold', fontsize = 16, color = 'black')
plt.xlabel('Presidential, Republican % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.ylabel('US Senate, Republican % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.xlim(10, 90)
plt.ylim(10, 90)
# plt.savefig('rep_pres_sen_10.png', bbox_inches='tight')
plt.plot();

In [ ]:
plt.figure(figsize = (8,8))
sns.set_theme(style="darkgrid")

x = all10['Dem%'].Pres
y = all10['Dem%'].US_Senate

graph = sns.regplot(data = all10, x=x, y=y, color='royalblue', scatter_kws={'s':40, 'edgecolor':'black'}) 
graph.axhline(50, color='red', linestyle = '--', linewidth = 2)
graph.axvline(50, color='red', linestyle = '--', linewidth = 2)

plt.title('Democratic Vote Share by County \n Presidential vs. US Senate, 2002-2008', fontweight = 'bold', fontsize = 16, color = 'black')
plt.xlabel('Presidential, Democratic % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.ylabel('US Senate, Democratic % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.xlim(10, 90)
plt.ylim(10, 90)
# plt.savefig('dem_pres_sen_10.png', bbox_inches='tight')
plt.plot();

In [ ]:
plt.figure(figsize = (8,8))
sns.set_theme(style="darkgrid")

x = all10['Rep%'].Pres
y = all10['Rep%'].US_House

graph = sns.regplot(data = all10, x=x, y=y, color='red', scatter_kws={'s':40, 'edgecolor':'black'}) 
graph.axhline(50, color='black', linestyle = '--', linewidth = 2)
graph.axvline(50, color='black', linestyle = '--', linewidth = 2)

plt.title('Republican Vote Share by County \n Presidential vs. US House, 2002-2008', fontweight = 'bold', fontsize = 16, color = 'black')
plt.xlabel('Presidential, Republican % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.ylabel('US House, Republican % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.xlim(10, 90)
plt.ylim(10, 90)
# plt.savefig('rep_pres_ush_10.png', bbox_inches='tight')
plt.plot();

In [ ]:
plt.figure(figsize = (8,8))
sns.set_theme(style="darkgrid")

x = all10['Dem%'].Pres
y = all10['Dem%'].US_House

graph = sns.regplot(data = all10, x=x, y=y, color='royalblue', scatter_kws={'s':40, 'edgecolor':'black'}) 
graph.axhline(50, color='red', linestyle = '--', linewidth = 2)
graph.axvline(50, color='red', linestyle = '--', linewidth = 2)

plt.title('Democratic Vote Share by County \n Presidential vs. US House, 2002-2008', fontweight = 'bold', fontsize = 16, color = 'black')
plt.xlabel('Presidential, Democratic % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.ylabel('US House, Democratic % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.xlim(10, 90)
plt.ylim(10, 90)
# plt.savefig('dem_pres_ush_10.png', bbox_inches='tight')
plt.plot();

In [ ]:
plt.figure(figsize = (8,8))
sns.set_theme(style="darkgrid")

x = all10['Rep%'].Pres
y = all10['Rep%'].State_House

graph = sns.regplot(data = all10, x=x, y=y, color='red', scatter_kws={'s':40, 'edgecolor':'black'}) 
graph.axhline(50, color='black', linestyle = '--', linewidth = 2)
graph.axvline(50, color='black', linestyle = '--', linewidth = 2)

plt.title('Republican Vote Share by County \n Presidential vs. TN House, 2002-2008', fontweight = 'bold', fontsize = 16, color = 'black')
plt.xlabel('Presidential, Republican % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.ylabel('TN House, Republican % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.xlim(10, 90)
plt.ylim(10, 90)
# plt.savefig('rep_pres_tnh_10.png', bbox_inches='tight')
plt.plot();

In [ ]:
plt.figure(figsize = (8,8))
sns.set_theme(style="darkgrid")

x = all10['Dem%'].Pres
y = all10['Dem%'].State_House

graph = sns.regplot(data = all10, x=x, y=y, color='royalblue', scatter_kws={'s':40, 'edgecolor':'black'}) 
graph.axhline(50, color='red', linestyle = '--', linewidth = 2)
graph.axvline(50, color='red', linestyle = '--', linewidth = 2)

plt.title('Democratic Vote Share by County \n Presidential vs. TN House, 2002-2008', fontweight = 'bold', fontsize = 16, color = 'black')
plt.xlabel('Presidential, Democratic % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.ylabel('TN House, Democratic % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.xlim(10, 90)
plt.ylim(10, 90)
# plt.savefig('dem_pres_tnh_10.png', bbox_inches='tight')
plt.plot();

In [ ]:
plt.figure(figsize = (8,8))
sns.set_theme(style="darkgrid")

x = all10['Rep%'].Governor
y = all10['Rep%'].US_Senate

graph = sns.regplot(data = all10, x=x, y=y, color='red', scatter_kws={'s':40, 'edgecolor':'black'}) 
graph.axhline(50, color='black', linestyle = '--', linewidth = 2)
graph.axvline(50, color='black', linestyle = '--', linewidth = 2)

plt.title('Republican Vote Share by County \n Gubernatorial vs. US Senate, 2002-2008', fontweight = 'bold', fontsize = 16, color = 'black')
plt.xlabel('Gubernatorial, Republican % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.ylabel('US Senate, Republican % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.xlim(10, 90)
plt.ylim(10, 90)
# plt.savefig('rep_gov_sen_10.png', bbox_inches='tight')
plt.plot();

In [ ]:
plt.figure(figsize = (8,8))
sns.set_theme(style="darkgrid")

x = all10['Dem%'].Governor
y = all10['Dem%'].US_Senate

graph = sns.regplot(data = all10, x=x, y=y, color='royalblue', scatter_kws={'s':40, 'edgecolor':'black'}) 
graph.axhline(50, color='red', linestyle = '--', linewidth = 2)
graph.axvline(50, color='red', linestyle = '--', linewidth = 2)

plt.title('Democratic Vote Share by County \n Gubernatorial vs. US Senate, 2002-2008', fontweight = 'bold', fontsize = 16, color = 'black')
plt.xlabel('Gubernatorial, Democratic % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.ylabel('US Senate, Democratic % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.xlim(10, 90)
plt.ylim(10, 90)
# plt.savefig('dem_gov_sen_10.png', bbox_inches='tight')
plt.plot();

In [ ]:
plt.figure(figsize = (8,8))
sns.set_theme(style="darkgrid")

x = all10['Rep%'].Governor
y = all10['Rep%'].US_House

graph = sns.regplot(data = all10, x=x, y=y, color='red', scatter_kws={'s':40, 'edgecolor':'black'}) 
graph.axhline(50, color='black', linestyle = '--', linewidth = 2)
graph.axvline(50, color='black', linestyle = '--', linewidth = 2)

plt.title('Republican Vote Share by County \n Gubernatorial vs. US House, 2002-2008', fontweight = 'bold', fontsize = 16, color = 'black')
plt.xlabel('Gubernatorial, Republican % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.ylabel('US House, Republican % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.xlim(10, 90)
plt.ylim(10, 90)
# plt.savefig('rep_gov_ush_10.png', bbox_inches='tight')
plt.plot();

In [ ]:
plt.figure(figsize = (8,8))
sns.set_theme(style="darkgrid")

x = all10['Dem%'].Governor
y = all10['Dem%'].US_House

graph = sns.regplot(data = all10, x=x, y=y, color='royalblue', scatter_kws={'s':40, 'edgecolor':'black'}) 
graph.axhline(50, color='red', linestyle = '--', linewidth = 2)
graph.axvline(50, color='red', linestyle = '--', linewidth = 2)

plt.title('Democratic Vote Share by County \n Gubernatorial vs. US House, 2002-2008', fontweight = 'bold', fontsize = 16, color = 'black')
plt.xlabel('Gubernatorial, Democratic % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.ylabel('US House, Democratic % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.xlim(10, 90)
plt.ylim(10, 90)
# plt.savefig('dem_gov_ush_10.png', bbox_inches='tight')
plt.plot();

In [ ]:
plt.figure(figsize = (8,8))
sns.set_theme(style="darkgrid")

x = all10['Rep%'].Governor
y = all10['Rep%'].State_House

graph = sns.regplot(data = all10, x=x, y=y, color='red', scatter_kws={'s':40, 'edgecolor':'black'}) 
graph.axhline(50, color='black', linestyle = '--', linewidth = 2)
graph.axvline(50, color='black', linestyle = '--', linewidth = 2)

plt.title('Republican Vote Share by County \n Gubernatorial vs. TN House, 2002-2008', fontweight = 'bold', fontsize = 16, color = 'black')
plt.xlabel('Gubernatorial, Republican % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.ylabel('TN House, Republican % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.xlim(10, 90)
plt.ylim(10, 90)
# plt.savefig('rep_gov_tnh_10.png', bbox_inches='tight')
plt.plot();

In [ ]:
plt.figure(figsize = (8,8))
sns.set_theme(style="darkgrid")

x = all10['Dem%'].Governor
y = all10['Dem%'].State_House

graph = sns.regplot(data = all10, x=x, y=y, color='royalblue', scatter_kws={'s':40, 'edgecolor':'black'}) 
graph.axhline(50, color='red', linestyle = '--', linewidth = 2)
graph.axvline(50, color='red', linestyle = '--', linewidth = 2)

plt.title('Democratic Vote Share by County \n Gubernatorial vs. TN House, 2002-2008', fontweight = 'bold', fontsize = 16, color = 'black')
plt.xlabel('Gubernatorial, Democratic % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.ylabel('TN House, Democratic % by County', fontweight = 'bold', fontsize = 14, color = 'black')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.xlim(10, 90)
plt.ylim(10, 90)
# plt.savefig('dem_gov_tnh_10.png', bbox_inches='tight')
plt.plot();

In [ ]:
all10.corr()

In [ ]:
all10_dem = all10['Dem%'][['Pres', 'Governor', 'US_Senate', 'US_House', 'State_House']]
all10_rep = all10['Rep%'][['Pres', 'Governor', 'US_Senate', 'US_House', 'State_House']]

In [ ]:
all10_dem.corr()

In [ ]:
all10_rep.corr()

In [ ]:
corr_d = all10_dem.corr()

f, ax = plt.subplots(figsize=(10, 6))
sns.heatmap(corr_d, cbar_kws = dict(location="right", shrink=0.8), cmap='Blues', annot=True, vmin=0.3,
           annot_kws={"size": 16, 'fontweight': 'bold'}, linewidths=0.3, linecolor='black', ax=ax)

ax.set_facecolor('dimgray')
f.set_facecolor('white')

plt.xlabel('Office', fontsize=16, fontweight='bold')
plt.ylabel('Office', fontsize=16, fontweight='bold')
plt.yticks(rotation=0, fontsize = 14)
plt.xticks(rotation=0, fontsize = 14)
plt.title('2002-2008 Correlation by Office \n Dem % by County', fontsize = 18, fontweight='bold', pad=20)

# plt.savefig('dem_county_corr_10.png', bbox_inches='tight')
plt.show();

In [ ]:
corr_r = all10_rep.corr()

f, ax = plt.subplots(figsize=(10, 6))
sns.heatmap(corr_r, cbar_kws = dict(location="right", shrink=0.8), cmap='Reds', annot=True, vmin=0.3,
           annot_kws={"size": 16, 'fontweight': 'bold'}, linewidths=0.3, linecolor='black', ax=ax)

ax.set_facecolor('dimgray')
f.set_facecolor('white')

plt.xlabel('Office', fontsize=16, fontweight='bold')
plt.ylabel('Office', fontsize=16, fontweight='bold')
plt.yticks(rotation=0, fontsize = 14)
plt.xticks(rotation=0, fontsize = 14)
plt.title('2002-2008 Correlation by Office \n Rep % by County', fontsize = 18, fontweight='bold', pad=20)

# plt.savefig('rep_county_corr_10.png', bbox_inches='tight')
plt.show();

In [ ]:
all20 = county_20[['year', 'county', 'office', 'Rep%', 'Dem%']]

In [ ]:
all20 = all20.pivot_table(index=['year', 'county'],
                 columns = 'office',
                 values = ['Rep%', 'Dem%']).reset_index()

In [ ]:
all20

In [ ]:
all20_dem = all20['Dem%'][['Pres', 'Governor', 'US_Senate', 'US_House', 'State_House']]
all20_rep = all20['Rep%'][['Pres', 'Governor', 'US_Senate', 'US_House', 'State_House']]

In [ ]:
all20_dem.corr()

In [ ]:
all20_rep.corr()

In [ ]:
corr_d = all20_dem.corr()
f, ax = plt.subplots(figsize=(10, 6))


sns.heatmap(corr_d, cbar_kws = dict(location="right", shrink=0.8), cmap='Blues', annot=True, vmin=0.3,
           annot_kws={"size": 16, 'fontweight': 'bold'}, linewidths=0.3, linecolor='black', ax=ax)

ax.set_facecolor('dimgray')
f.set_facecolor('white')

plt.xlabel('Office', fontsize=16, fontweight='bold')
plt.ylabel('Office', fontsize=16, fontweight='bold')
plt.yticks(rotation=0, fontsize = 14)
plt.xticks(rotation=0, fontsize = 14)
plt.title('2010-2020 Correlation by Office \n Dem % by County', fontsize = 18, fontweight='bold', pad=20)

# plt.savefig('dem_county_corr_20.png', bbox_inches='tight')
plt.show();

In [ ]:
corr_r = all20_rep.corr()

f, ax = plt.subplots(figsize=(10, 6))
sns.heatmap(corr_r, cbar_kws = dict(location="right", shrink=0.8), cmap='Reds', annot=True, vmin=0.3,
           annot_kws={"size": 16, 'fontweight': 'bold'}, linewidths=0.3, linecolor='black', ax=ax)

ax.set_facecolor('dimgray')
f.set_facecolor('white')

plt.xlabel('Office', fontsize=16, fontweight='bold')
plt.ylabel('Office', fontsize=16, fontweight='bold')
plt.yticks(rotation=0, fontsize = 14)
plt.xticks(rotation=0, fontsize = 14)
plt.title('2010-2020 Correlation by Office \n Rep % by County', fontsize = 18, fontweight='bold', pad=20)

# plt.savefig('rep_county_corr_20.png', bbox_inches='tight')
plt.show();